In [3]:
import os
os.listdir('./raw_data')

['낙농_ICT데이터_ICT착유기정보.xlsx',
 '.ipynb_checkpoints',
 '낙농_개체별_번식이력.xlsx',
 '낙농_ICT데이터_환경관리기정보.xlsx',
 '낙농_ICT데이터_착유군사급이기정보.xlsx',
 '낙농_ICT데이터_체중정보.xlsx',
 '낙농_ICT데이터_군사급이기정보.xlsx',
 '로봇착유기',
 '낙농_ICT데이터_로봇착유기정보.xlsx',
 '낙농_ICT데이터_환경제어기정보.xlsx',
 '낙농_월별_번식성적.xlsx',
 '낙농_ICT데이터_TMR급이기정보.xlsx',
 'ICT_착유기',
 '낙농_ICT데이터_일반급이기정보.xlsx']

### target data

예측에 target이 되는 체중 정보에 대해서 review

In [9]:
import pandas as pd

weight_data = pd.read_excel('./raw_data/낙농_ICT데이터_체중정보.xlsx', header=1)
weight_data

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,체중(kg)
0,1,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-11 00:00,761.0
1,2,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-10 00:00,751.0
2,3,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-09 00:00,749.0
3,4,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-08 00:00,750.0
4,5,4.100000e+09,200~300,20278,20120112010022,(주)애그리로보텍,1,2020-09-07 00:00,750.0
...,...,...,...,...,...,...,...,...,...
59371,59372,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-22 00:00,622.0
59372,59373,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-21 00:00,620.0
59373,59374,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-20 00:00,570.0
59374,59375,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-19 00:00,626.0


In [10]:
weight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59376 entries, 0 to 59375
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No      59376 non-null  int64  
 1   지역      48163 non-null  float64
 2   규모      59376 non-null  object 
 3   농가명     59376 non-null  int64  
 4   개체번호    59376 non-null  int64  
 5   제조사아이디  59376 non-null  object 
 6   장비번호    59376 non-null  int64  
 7   측정일시    59376 non-null  object 
 8   체중(kg)  59376 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 4.1+ MB


In [18]:
# 총 농가수 확인
weight_data['농가명'].unique()

array([20278, 20264, 20279, 21133])

In [19]:
# 총 개체(소)수 확인
len(weight_data['개체번호'].unique()) , len(weight_data['개체번호'].unique()) // 4

(310, 77)

In [22]:
count_by_entity = weight_data.groupby('개체번호')['측정일시'].count()
count_by_entity

개체번호
20071210010043     16
20091228020038    289
20100210010017    252
20110421010013    190
20110425020096    300
                 ... 
20200128010113     93
20200129010114     38
20200130010115     81
20200202010116     77
20200306010117     32
Name: 측정일시, Length: 310, dtype: int64

In [24]:
# 개채번호별 row 수 확인
count_by_entity = weight_data.groupby('개체번호')['측정일시'].count().reset_index()
count_by_entity.columns = ['개체번호', '측정일시_개수']

count_by_entity

,개체번호,측정일시_개수
0,20071210010043,16
1,20091228020038,289
2,20100210010017,252
3,20110421010013,190
4,20110425020096,300
...,...,...
305,20200128010113,93
306,20200129010114,38
307,20200130010115,81
308,20200202010116,77


In [27]:
# 이를 histogram로 표시 
import plotly.express as px
fig = px.histogram(count_by_entity, x='측정일시_개수', nbins=30, labels={'측정일시_개수':'측정일시 개수'})
fig.show()

In [35]:
weight_data['측정일시'] = pd.to_datetime(weight_data['측정일시'])
weight_data['측정일시'] = weight_data['측정일시'].apply(lambda x: x.replace(minute=0, second=0))
weight_data['측정일시'].unique()


array(['2020-09-11T00:00:00.000000000', '2020-09-10T00:00:00.000000000',
       '2020-09-09T00:00:00.000000000', '2020-09-08T00:00:00.000000000',
       '2020-09-07T00:00:00.000000000', '2020-09-06T00:00:00.000000000',
       '2020-09-05T00:00:00.000000000', '2020-09-04T00:00:00.000000000',
       '2020-09-03T00:00:00.000000000', '2020-09-02T00:00:00.000000000',
       '2020-09-01T00:00:00.000000000', '2020-08-31T00:00:00.000000000',
       '2020-08-30T00:00:00.000000000', '2020-08-29T00:00:00.000000000',
       '2020-08-28T00:00:00.000000000', '2020-08-27T00:00:00.000000000',
       '2020-08-26T00:00:00.000000000', '2020-08-25T00:00:00.000000000',
       '2020-08-24T00:00:00.000000000', '2020-08-23T00:00:00.000000000',
       '2020-08-22T00:00:00.000000000', '2020-08-21T00:00:00.000000000',
       '2020-08-20T00:00:00.000000000', '2020-08-19T00:00:00.000000000',
       '2020-08-18T00:00:00.000000000', '2020-08-17T00:00:00.000000000',
       '2020-08-16T00:00:00.000000000', '2020-08-15

In [36]:
weight_data = weight_data.sort_values(by=['개체번호', '측정일시'], ascending=[True, False])
weight_data

,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,체중(kg)
48163,48164,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-11-04,0.0
48164,48165,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-31,0.0
48165,48166,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-25,757.0
48166,48167,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-24,773.0
48167,48168,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-23,773.0
...,...,...,...,...,...,...,...,...,...
59371,59372,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-22,622.0
59372,59373,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-21,620.0
59373,59374,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-20,570.0
59374,59375,NaN,200~300,21133,20200306010117,(주)애그리로보텍,1,2022-02-19,626.0


In [46]:
cow_id = weight_data['개체번호'].unique()[0]
cow_data = weight_data[weight_data['개체번호'] == cow_id]
cow_data

,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,체중(kg)
48163,48164,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-11-04,0.0
48164,48165,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-31,0.0
48165,48166,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-25,757.0
48166,48167,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-24,773.0
48167,48168,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-23,773.0
48168,48169,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-22,759.0
48169,48170,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-21,751.0
48170,48171,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-20,768.0
48171,48172,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-19,738.0
48172,48173,NaN,200~300,21133,20071210010043,(주)애그리로보텍,1,2021-10-18,762.0


In [67]:
# 기록이 되어 있는 timestamp 확인
dt = pd.date_range(weight_data['측정일시'].min(), weight_data['측정일시'].max(), freq='D')
unique_cow_ids = weight_data['개체번호'].unique()
new_dataframe = pd.DataFrame(index=dt, columns=unique_cow_ids)

for index, row in weight_data.iterrows():
    new_dataframe.loc[row['측정일시'], row['개체번호']] = 1
new_dataframe = new_dataframe.fillna(0)
new_dataframe

,20071210010043,20091228020038,20100210010017,20110421010013,20110425020096,20110607010014,20110702010124,20120112010022,20120410020064,20120423010010,...,20191122010167,20191207010168,20200103010109,20200103010110,20200127010111,20200128010113,20200129010114,20200130010115,20200202010116,20200306010117
2020-01-01,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-05,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-17,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2022-03-18,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2022-03-19,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2022-03-20,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [69]:
for column in new_dataframe.columns:
    consecutive_groups = []
    current_group = []

    for date in new_dataframe.index:
        if new_dataframe.at[date, column] == 1:
            current_group.append(date)
        else:
            if current_group:
                consecutive_groups.append(current_group)
                current_group = []

    if current_group:
        consecutive_groups.append(current_group)

    for group in consecutive_groups:
        if len(group) > 1:
            print(f"column {column}: start {group[0]}, end {group[-1]}")

column 20071210010043: start 2021-10-12 00:00:00, end 2021-10-25 00:00:00
column 20091228020038: start 2020-01-12 00:00:00, end 2020-10-26 00:00:00
column 20100210010017: start 2020-01-01 00:00:00, end 2020-04-01 00:00:00
column 20100210010017: start 2020-04-03 00:00:00, end 2020-08-02 00:00:00
column 20100210010017: start 2020-08-06 00:00:00, end 2020-08-17 00:00:00
column 20100210010017: start 2020-08-23 00:00:00, end 2020-08-25 00:00:00
column 20100210010017: start 2020-08-28 00:00:00, end 2020-09-11 00:00:00
column 20100210010017: start 2020-09-16 00:00:00, end 2020-09-23 00:00:00
column 20110421010013: start 2020-01-01 00:00:00, end 2020-07-07 00:00:00
column 20110425020096: start 2020-01-01 00:00:00, end 2020-10-26 00:00:00
column 20110607010014: start 2020-01-22 00:00:00, end 2020-10-26 00:00:00
column 20110702010124: start 2020-01-01 00:00:00, end 2020-06-03 00:00:00
column 20120112010022: start 2020-01-21 00:00:00, end 2020-09-11 00:00:00
column 20120410020064: start 2020-01-0

column 20150527010041: start 2020-01-21 00:00:00, end 2020-02-19 00:00:00
column 20150527010041: start 2020-02-21 00:00:00, end 2020-08-09 00:00:00
column 20150527010041: start 2020-10-23 00:00:00, end 2020-10-26 00:00:00
column 20150710010052: start 2021-10-27 00:00:00, end 2021-10-31 00:00:00
column 20150710010052: start 2021-11-02 00:00:00, end 2021-11-09 00:00:00
column 20150710010052: start 2021-11-11 00:00:00, end 2021-11-14 00:00:00
column 20150710010052: start 2021-11-16 00:00:00, end 2021-11-21 00:00:00
column 20150710010052: start 2021-11-23 00:00:00, end 2021-12-05 00:00:00
column 20150710010052: start 2021-12-07 00:00:00, end 2021-12-10 00:00:00
column 20150710010052: start 2021-12-12 00:00:00, end 2021-12-17 00:00:00
column 20150710010052: start 2021-12-19 00:00:00, end 2021-12-21 00:00:00
column 20150710010052: start 2021-12-23 00:00:00, end 2021-12-25 00:00:00
column 20150710010052: start 2022-01-02 00:00:00, end 2022-01-04 00:00:00
column 20150710010052: start 2022-01-1

column 20170802010006: start 2020-04-06 00:00:00, end 2020-10-26 00:00:00
column 20170804020115: start 2020-01-01 00:00:00, end 2020-06-23 00:00:00
column 20170804020115: start 2020-08-16 00:00:00, end 2020-10-26 00:00:00
column 20170805010069: start 2021-10-19 00:00:00, end 2021-10-31 00:00:00
column 20170805010069: start 2021-11-02 00:00:00, end 2021-11-09 00:00:00
column 20170805010069: start 2021-11-11 00:00:00, end 2021-11-14 00:00:00
column 20170805010069: start 2021-11-16 00:00:00, end 2021-11-21 00:00:00
column 20170805010069: start 2021-11-23 00:00:00, end 2021-12-05 00:00:00
column 20170805010069: start 2021-12-07 00:00:00, end 2021-12-10 00:00:00
column 20170805010069: start 2021-12-12 00:00:00, end 2021-12-17 00:00:00
column 20170805010069: start 2021-12-19 00:00:00, end 2021-12-21 00:00:00
column 20170805010069: start 2021-12-23 00:00:00, end 2021-12-25 00:00:00
column 20170805010069: start 2022-01-02 00:00:00, end 2022-01-04 00:00:00
column 20170805010069: start 2022-01-1

column 20180605010025: start 2021-12-07 00:00:00, end 2021-12-10 00:00:00
column 20180605010025: start 2021-12-12 00:00:00, end 2021-12-17 00:00:00
column 20180605010025: start 2021-12-19 00:00:00, end 2021-12-21 00:00:00
column 20180605010025: start 2021-12-23 00:00:00, end 2021-12-25 00:00:00
column 20180605010025: start 2022-01-02 00:00:00, end 2022-01-04 00:00:00
column 20180605010025: start 2022-01-10 00:00:00, end 2022-01-13 00:00:00
column 20180605010025: start 2022-01-15 00:00:00, end 2022-01-22 00:00:00
column 20180605010025: start 2022-01-24 00:00:00, end 2022-02-07 00:00:00
column 20180605010025: start 2022-02-14 00:00:00, end 2022-03-21 00:00:00
column 20180613010064: start 2020-08-08 00:00:00, end 2020-10-26 00:00:00
column 20180614010084: start 2020-08-31 00:00:00, end 2020-10-26 00:00:00
column 20180614020078: start 2020-07-07 00:00:00, end 2020-10-26 00:00:00
column 20180618010065: start 2020-01-01 00:00:00, end 2020-10-26 00:00:00
column 20180620010009: start 2020-08-1

column 20200103010110: start 2021-12-19 00:00:00, end 2021-12-21 00:00:00
column 20200103010110: start 2021-12-23 00:00:00, end 2021-12-25 00:00:00
column 20200103010110: start 2022-01-02 00:00:00, end 2022-01-04 00:00:00
column 20200103010110: start 2022-01-10 00:00:00, end 2022-01-13 00:00:00
column 20200103010110: start 2022-01-15 00:00:00, end 2022-01-22 00:00:00
column 20200103010110: start 2022-01-24 00:00:00, end 2022-02-07 00:00:00
column 20200103010110: start 2022-02-14 00:00:00, end 2022-03-21 00:00:00
column 20200127010111: start 2021-12-12 00:00:00, end 2021-12-17 00:00:00
column 20200127010111: start 2021-12-19 00:00:00, end 2021-12-21 00:00:00
column 20200127010111: start 2021-12-23 00:00:00, end 2021-12-25 00:00:00
column 20200127010111: start 2022-01-02 00:00:00, end 2022-01-04 00:00:00
column 20200127010111: start 2022-01-10 00:00:00, end 2022-01-13 00:00:00
column 20200127010111: start 2022-01-15 00:00:00, end 2022-01-22 00:00:00
column 20200127010111: start 2022-01-2

In [72]:
result_dataframe = pd.DataFrame(columns=['개체번호', 'start', 'end'])

for column in new_dataframe.columns:
    consecutive_groups = []
    current_group = []

    for date in new_dataframe.index:
        if new_dataframe.at[date, column] == 1:
            current_group.append(date)
        else:
            if current_group:
                consecutive_groups.append(current_group)
                current_group = []

    if current_group:
        consecutive_groups.append(current_group)

    for group in consecutive_groups:
        if len(group) > 1:
            result_dataframe = result_dataframe.append({
                '개체번호': column,
                'start': group[0],
                'end': group[-1]
            }, ignore_index=True)

result_dataframe

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

...

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-72-eb5c27ca4cb6>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

,개체번호,start,end
0,20071210010043,2021-10-12,2021-10-25
1,20091228020038,2020-01-12,2020-10-26
2,20100210010017,2020-01-01,2020-04-01
3,20100210010017,2020-04-03,2020-08-02
4,20100210010017,2020-08-06,2020-08-17
...,...,...,...
1492,20200202010116,2022-01-10,2022-01-13
1493,20200202010116,2022-01-15,2022-01-22
1494,20200202010116,2022-01-24,2022-02-07
1495,20200202010116,2022-02-14,2022-03-21


In [80]:
# 연속된 timestmap 및 차이 확인
dt = pd.date_range(weight_data['측정일시'].min(), weight_data['측정일시'].max(), freq='D')

# 개체번호 고유값들 가져오기
unique_cow_ids = weight_data['개체번호'].unique()

# 새로운 데이터프레임 생성
new_dataframe = pd.DataFrame(index=dt, columns=unique_cow_ids)

# 각 날짜와 개체번호에 해당하는 값을 원핫 인코딩하여 입력
for index, row in weight_data.iterrows():
    new_dataframe.loc[row['측정일시'], row['개체번호']] = 1

# NaN 값을 0으로 채우기
new_dataframe = new_dataframe.fillna(0)

# 각 컬럼별로 연속된 값의 시작과 끝 정보 및 차이를 저장하는 데이터프레임 생성
result_dataframe = pd.DataFrame(columns=['컬럼', '시작', '끝', '차이'])

for column in new_dataframe.columns:
    consecutive_groups = []
    current_group = []

    for date in new_dataframe.index:
        if new_dataframe.at[date, column] == 1:
            current_group.append(date)
        else:
            if current_group:
                consecutive_groups.append(current_group)
                current_group = []

    if current_group:
        consecutive_groups.append(current_group)

    for group in consecutive_groups:
        if len(group) > 1:
            difference = (group[-1] - group[0]).days
            result_dataframe = result_dataframe.append({
                '컬럼': column,
                '시작': group[0],
                '끝': group[-1],
                '차이': difference
            }, ignore_index=True)

result_dataframe

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

...

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-80-956fa31c66d2>:37: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

,컬럼,시작,끝,차이
0,20071210010043,2021-10-12,2021-10-25,13
1,20091228020038,2020-01-12,2020-10-26,288
2,20100210010017,2020-01-01,2020-04-01,91
3,20100210010017,2020-04-03,2020-08-02,121
4,20100210010017,2020-08-06,2020-08-17,11
...,...,...,...,...
1492,20200202010116,2022-01-10,2022-01-13,3
1493,20200202010116,2022-01-15,2022-01-22,7
1494,20200202010116,2022-01-24,2022-02-07,14
1495,20200202010116,2022-02-14,2022-03-21,35


In [83]:
result_dataframe[result_dataframe['차이'] > 100]

,컬럼,시작,끝,차이
1,20091228020038,2020-01-12,2020-10-26,288
3,20100210010017,2020-04-03,2020-08-02,121
8,20110421010013,2020-01-01,2020-07-07,188
9,20110425020096,2020-01-01,2020-10-26,299
10,20110607010014,2020-01-22,2020-10-26,278
...,...,...,...,...
936,20180726010004,2020-07-03,2020-10-26,115
972,20180802020063,2020-06-24,2020-10-26,124
974,20180806010006,2020-05-07,2020-10-26,172
975,20180807010062,2020-01-18,2020-10-26,282


In [84]:
result_dataframe[result_dataframe['차이'] > 200]

,컬럼,시작,끝,차이
1,20091228020038,2020-01-12,2020-10-26,288
9,20110425020096,2020-01-01,2020-10-26,299
10,20110607010014,2020-01-22,2020-10-26,278
12,20120112010022,2020-01-21,2020-09-11,234
14,20120410020064,2020-02-21,2020-10-26,248
...,...,...,...,...
829,20180427020022,2020-03-08,2020-10-26,232
855,20180603010001,2020-01-12,2020-10-26,288
873,20180618010065,2020-01-01,2020-10-26,299
919,20180717010019,2020-01-01,2020-10-26,299


In [85]:
result_dataframe[result_dataframe['차이'] > 250]

,컬럼,시작,끝,차이
1,20091228020038,2020-01-12,2020-10-26,288
9,20110425020096,2020-01-01,2020-10-26,299
10,20110607010014,2020-01-22,2020-10-26,278
15,20121004020089,2020-01-21,2020-10-26,279
18,20130102020043,2020-01-20,2020-10-26,280
...,...,...,...,...
796,20180315010059,2020-02-14,2020-10-26,255
855,20180603010001,2020-01-12,2020-10-26,288
873,20180618010065,2020-01-01,2020-10-26,299
919,20180717010019,2020-01-01,2020-10-26,299


In [86]:
result_dataframe[result_dataframe['차이'] > 300]

,컬럼,시작,끝,차이


In [107]:
weight_id = weight_data['개체번호'].unique()
weight_id

array([20071210010043, 20091228020038, 20100210010017, 20110421010013,
       20110425020096, 20110607010014, 20110702010124, 20120112010022,
       20120410020064, 20120423010010, 20121004020089, 20130101010002,
       20130102020043, 20130126010057, 20130201010003, 20130528010023,
       20130609010024, 20130708010010, 20130816010079, 20130907020069,
       20131019010013, 20131205020084, 20140113010011, 20140225010046,
       20140315020075, 20140417010025, 20140614010003, 20140810010048,
       20140810020030, 20140820020024, 20140909010008, 20140913010005,
       20140918010026, 20140925010027, 20140930010028, 20140930010029,
       20141014010030, 20141014010043, 20141016020060, 20141030010031,
       20141102010013, 20141102020021, 20141110010058, 20141119010026,
       20141119010046, 20141204020102, 20141230010050, 20150114010059,
       20150121010025, 20150126010004, 20150128010060, 20150203010044,
       20150216010014, 20150218010032, 20150220010016, 20150223010020,
      

### open api에서 출생일자를 받아서 이를 이용해서 time_idx 부여

### 착유기 정보
ICT 착유기 및 로봇 착유기가 있음 

In [92]:
# ICT 착유기
import os
import pandas as pd
from tqdm import tqdm_notebook

directory = './raw_data/ICT_착유기'
file_list = os.listdir(directory)
excel_files = [file for file in file_list if file.endswith('.xlsx')]

combined_data = pd.DataFrame()

for excel_file in tqdm_notebook(excel_files):
    file_path = os.path.join(directory, excel_file)
    data = pd.read_excel(file_path)
    combined_data = combined_data.append(data, ignore_index=True)

combined_data


<ipython-input-92-7e32806522a6>:13: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/34 [00:00<?, ?it/s]

<ipython-input-92-7e32806522a6>:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-92-7e32806522a6>:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-92-7e32806522a6>:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-inpu

<ipython-input-92-7e32806522a6>:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-92-7e32806522a6>:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-92-7e32806522a6>:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-inpu

,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,착유량(L),착유시작일시,착유종료일시,착유시간,착유회차,전도도
0,1,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-07 04:46,17,2022-12-07 04:46,2022-12-07 05:09,23:01,1.0,0.0
1,2,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-06 04:29,17,2022-12-06 04:29,2022-12-06 04:39,10:57,1.0,0.0
2,3,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-05 04:39,17,2022-12-05 04:39,2022-12-05 04:50,11:44,1.0,0.0
3,4,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-02 04:35,16,2022-12-02 04:35,2022-12-02 04:43,08:34,1.0,0.0
4,5,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-11-17 04:15,16,2022-11-17 04:15,2022-11-17 04:38,23:16,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482116,64956,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-03 05:19,18,NaN,NaN,NaN,NaN,10.3
2482117,64957,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-02 17:01,17,NaN,NaN,NaN,NaN,11.1
2482118,64958,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-02 05:04,19,NaN,NaN,NaN,NaN,9.3
2482119,64959,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-01 16:48,18,NaN,NaN,NaN,NaN,8.6


In [99]:
# data merge
combined_data.to_csv('./raw_data/낙농_ICT데이터_ICT착유기정보.csv')

In [100]:
milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_ICT착유기정보.csv')
milk_data

,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,착유량(L),착유시작일시,착유종료일시,착유시간,착유회차,전도도
0,0,1,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-07 04:46,17,2022-12-07 04:46,2022-12-07 05:09,23:01,1.0,0.0
1,1,2,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-06 04:29,17,2022-12-06 04:29,2022-12-06 04:39,10:57,1.0,0.0
2,2,3,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-05 04:39,17,2022-12-05 04:39,2022-12-05 04:50,11:44,1.0,0.0
3,3,4,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-12-02 04:35,16,2022-12-02 04:35,2022-12-02 04:43,08:34,1.0,0.0
4,4,5,4.100000e+09,300~400,20446,20110405010031,드라발 (주),1,2022-11-17 04:15,16,2022-11-17 04:15,2022-11-17 04:38,23:16,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482116,2482116,64956,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-03 05:19,18,NaN,NaN,NaN,NaN,10.3
2482117,2482117,64957,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-02 17:01,17,NaN,NaN,NaN,NaN,11.1
2482118,2482118,64958,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-02 05:04,19,NaN,NaN,NaN,NaN,9.3
2482119,2482119,64959,NaN,500~1000,20259,20190227010163,FULLWOOD,1,2021-07-01 16:48,18,NaN,NaN,NaN,NaN,8.6


In [110]:
milk_data = milk_data[milk_data['개체번호'].isin(weight_id)]
milk_data

,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,착유량(L),착유시작일시,착유종료일시,착유시간,착유회차,전도도
22551,22551,22552,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2022-12-31 17:47,3,2022-12-31 17:47,2022-12-31 17:50,03:05,2.0,8.7
22552,22552,22553,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2022-12-31 06:01,6,2022-12-31 06:01,2022-12-31 06:13,12:34,1.0,9.2
22553,22553,22554,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2022-12-30 17:57,4,2022-12-30 17:57,2022-12-30 18:02,05:07,2.0,8.7
22554,22554,22555,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2022-12-30 06:06,5,2022-12-30 06:06,2022-12-30 06:11,05:00,1.0,8.3
22555,22555,22556,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2022-12-29 17:44,4,2022-12-29 17:44,2022-12-29 17:48,04:35,2.0,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2440907,2440907,23747,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2021-07-15 06:00,17,2021-07-15 06:00,2021-07-15 06:04,04:19,1.0,7.5
2440908,2440908,23748,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2021-07-14 17:34,16,2021-07-14 17:34,2021-07-14 17:45,11:47,2.0,7.7
2440909,2440909,23749,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2021-07-14 06:02,18,2021-07-14 06:02,2021-07-14 06:06,04:03,1.0,7.7
2440910,2440910,23750,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2021-07-13 17:42,17,2021-07-13 17:42,2021-07-13 17:46,04:53,2.0,7.9


In [111]:
milk_data['개체번호'].unique()

array([20160819010038])

In [121]:
# ICT 착유기
import os
import pandas as pd
from tqdm import tqdm_notebook

directory = './raw_data/로봇_착유기'
file_list = os.listdir(directory)
excel_files = [file for file in file_list if file.endswith('.xlsx')]

combined_data = pd.DataFrame()

for excel_file in tqdm_notebook(excel_files):
    file_path = os.path.join(directory, excel_file)
    data = pd.read_excel(file_path , header=1)
    combined_data = combined_data.append(data, ignore_index=True)

combined_data
combined_data.to_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')

<ipython-input-121-33538ebf1219>:12: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/13 [00:00<?, ?it/s]

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-121-33538ebf1219>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-121-33538ebf1219>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-121-33538ebf1219>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/npla

In [122]:
milk_data = pd.read_csv('./raw_data/낙농_ICT데이터_로봇착유기정보.csv')
milk_data

<ipython-input-122-d85304f20f8e>:1: DtypeWarning:

Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,착유량(L),...,전도도,체세포,혈액흐름,색상,온도(℃),유지방,유단백,반추시간,공기흐름,산차
0,0,1,4.100000e+09,100두 미만,20336,20130111020015,(주)애그리로보텍,1,2020-05-31 11:17,24,...,7.00,NaN,N,NaN,39.8,0.00,0.00,NaN,4.9,NaN
1,1,2,4.100000e+09,100두 미만,20336,20130111020015,(주)애그리로보텍,1,2020-05-30 22:39,24,...,7.10,NaN,N,NaN,39.3,0.00,0.00,NaN,5.3,NaN
2,2,3,4.100000e+09,100두 미만,20336,20130111020015,(주)애그리로보텍,1,2020-05-30 11:30,23,...,7.00,NaN,N,NaN,39.5,0.00,0.00,NaN,5.7,NaN
3,3,4,4.100000e+09,100두 미만,20336,20130111020015,(주)애그리로보텍,1,2020-05-30 01:31,23,...,7.10,NaN,N,NaN,39.2,0.00,0.00,NaN,5.0,NaN
4,4,5,4.100000e+09,100두 미만,20336,20130111020015,(주)애그리로보텍,1,2020-05-29 14:26,16,...,7.10,NaN,N,NaN,39.6,0.00,0.00,NaN,4.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822735,822735,9661,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-07 00:00,18,...,71.75,793.0,NaN,NaN,NaN,4.97,3.70,566.0,NaN,1.0
822736,822736,9662,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-06 00:00,16,...,76.50,NaN,NaN,NaN,NaN,5.92,3.81,433.0,NaN,1.0
822737,822737,9663,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-05 00:00,12,...,76.50,NaN,NaN,NaN,NaN,6.85,4.19,328.0,NaN,1.0
822738,822738,9664,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-04 00:00,9,...,80.00,NaN,NaN,NaN,NaN,7.50,4.62,301.0,NaN,1.0


In [124]:
milk_data = milk_data[milk_data['개체번호'].isin(weight_id)]
milk_data

,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,착유량(L),...,전도도,체세포,혈액흐름,색상,온도(℃),유지방,유단백,반추시간,공기흐름,산차
443,443,444,NaN,100~200,20264,20091228020038,(주)애그리로보텍,1,2020-05-31 00:00,37,...,55.00,59.0,NaN,NaN,NaN,3.58,3.21,655.0,NaN,8.0
444,444,445,NaN,100~200,20264,20091228020038,(주)애그리로보텍,1,2020-05-30 00:00,38,...,55.75,83.0,NaN,NaN,NaN,3.85,3.13,647.0,NaN,8.0
445,445,446,NaN,100~200,20264,20091228020038,(주)애그리로보텍,1,2020-05-29 00:00,30,...,54.00,69.0,NaN,NaN,NaN,4.00,3.13,674.0,NaN,8.0
446,446,447,NaN,100~200,20264,20091228020038,(주)애그리로보텍,1,2020-05-28 00:00,38,...,55.00,69.0,NaN,NaN,NaN,3.96,3.13,670.0,NaN,8.0
447,447,448,NaN,100~200,20264,20091228020038,(주)애그리로보텍,1,2020-05-27 00:00,35,...,54.50,67.0,NaN,NaN,NaN,3.93,3.14,626.0,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822735,822735,9661,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-07 00:00,18,...,71.75,793.0,NaN,NaN,NaN,4.97,3.70,566.0,NaN,1.0
822736,822736,9662,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-06 00:00,16,...,76.50,NaN,NaN,NaN,NaN,5.92,3.81,433.0,NaN,1.0
822737,822737,9663,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-05 00:00,12,...,76.50,NaN,NaN,NaN,NaN,6.85,4.19,328.0,NaN,1.0
822738,822738,9664,NaN,100~200,20278,20180327010075,(주)애그리로보텍,1,2020-03-04 00:00,9,...,80.00,NaN,NaN,NaN,NaN,7.50,4.62,301.0,NaN,1.0


In [126]:
len(milk_data['개체번호'].unique())

310

In [127]:
milk_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343014 entries, 443 to 822739
Data columns (total 23 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  343014 non-null  int64  
 1   No          343014 non-null  int64  
 2   지역          0 non-null       float64
 3   규모          343014 non-null  object 
 4   농가명         343014 non-null  int64  
 5   개체번호        343014 non-null  int64  
 6   제조사아이디      343014 non-null  object 
 7   장비번호        343014 non-null  int64  
 8   측정일시        343014 non-null  object 
 9   착유량(L)      343014 non-null  int64  
 10  착유시작일시      343014 non-null  object 
 11  착유종료일시      343014 non-null  object 
 12  착유회차        265714 non-null  float64
 13  전도도         343014 non-null  float64
 14  체세포         79696 non-null   float64
 15  혈액흐름        265714 non-null  object 
 16  색상          4107 non-null    float64
 17  온도(℃)       265713 non-null  float64
 18  유지방         337961 non-null  float64
 19  

### 번식 이력 

In [128]:
breeding_data = pd.read_excel('./raw_data/낙농_개체별_번식이력.xlsx' , header=1)


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [129]:
breeding_data

,No,지역,규모,농가명,개체번호,산차,수정차수,수정일,수정유형,정액코드,건유일,분만일,임신기간(일),종부횟수,공태일수,분만간격(일),초산일령
0,1,3.100000e+09,300~400,20276,20141023010133,5,1,2022-01-22,인공수정,7HO13930,NaN,NaN,NaN,1,NaN,NaN,NaN
1,2,3.100000e+09,300~400,20276,20141023010133,4,3,2021-03-30,인공수정,7HO12796,2021-10-31,2021-12-30,275.0,3,583.0,858.0,NaN
2,3,3.100000e+09,300~400,20276,20141023010133,4,2,2020-04-29,인공수정,7HO12206,NaN,NaN,NaN,2,NaN,NaN,NaN
3,4,3.100000e+09,300~400,20276,20141023010133,4,1,2019-10-20,인공수정,7HO11316,NaN,NaN,NaN,1,NaN,NaN,NaN
4,5,3.100000e+09,300~400,20276,20141023010133,3,2,2018-11-19,인공수정,250HO11837,2019-06-25,2019-08-25,279.0,2,87.0,366.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31110,31111,NaN,500~1000,20249,20210802020406,0,1,2022-09-30,인공수정,200HO10845,2022-10-22,2022-11-23,54.0,1,NaN,NaN,479.0
31111,31112,NaN,500~1000,20249,20210809020404,0,1,2022-09-27,인공수정,200HO10845,2022-10-22,2022-11-23,57.0,1,NaN,NaN,472.0
31112,31113,NaN,500~1000,20249,20210815020403,0,1,2022-11-11,인공수정,200HO10845,2022-10-22,2022-11-23,12.0,1,NaN,NaN,466.0
31113,31114,NaN,500~1000,20249,20210901020402,0,1,2022-11-11,인공수정,203HO01468,2022-10-22,2022-11-23,12.0,1,NaN,NaN,449.0


In [130]:
breeding_data = breeding_data[breeding_data['개체번호'].isin(weight_id)]
breeding_data

,No,지역,규모,농가명,개체번호,산차,수정차수,수정일,수정유형,정액코드,건유일,분만일,임신기간(일),종부횟수,공태일수,분만간격(일),초산일령
6428,6429,4.400000e+09,300~400,21678,20160819010038,3,1,2021-03-04,인공수정,534HO00071,2021-10-06,2021-12-01,272.0,1,159.0,431.0,NaN
6429,6430,4.400000e+09,300~400,21678,20160819010038,2,1,2019-12-23,인공수정,1HO11048,2020-07-20,2020-09-26,278.0,1,96.0,374.0,NaN
6430,6431,4.400000e+09,300~400,21678,20160819010038,1,1,2018-12-13,인공수정,1HO09527,2019-07-19,2019-09-18,279.0,1,129.0,408.0,NaN
6431,6432,4.400000e+09,300~400,21678,20160819010038,0,1,2017-11-10,인공수정,1HO09828,NaN,2018-08-06,269.0,1,NaN,NaN,718.0
13790,13791,NaN,100~200,20281,20161015010052,1,4,2020-06-30,인공수정,7HO14477,NaN,NaN,NaN,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25553,25554,NaN,500~1000,21144,20180322010007,1,4,2021-05-16,인공수정,KPN,NaN,NaN,NaN,4,NaN,NaN,NaN
25554,25555,NaN,500~1000,21144,20180322010007,1,3,2021-04-20,인공수정,208HO00344,NaN,NaN,NaN,3,NaN,NaN,NaN
25555,25556,NaN,500~1000,21144,20180322010007,1,2,2021-03-23,인공수정,208HO00344,NaN,NaN,NaN,2,NaN,NaN,NaN
25556,25557,NaN,500~1000,21144,20180322010007,1,1,2020-08-26,인공수정,208HO00344,NaN,NaN,NaN,1,NaN,NaN,NaN


In [132]:
breeding_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1444 entries, 6428 to 25557
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       1444 non-null   int64  
 1   지역       4 non-null      float64
 2   규모       1444 non-null   object 
 3   농가명      1444 non-null   int64  
 4   개체번호     1444 non-null   int64  
 5   산차       1444 non-null   int64  
 6   수정차수     1444 non-null   int64  
 7   수정일      1444 non-null   object 
 8   수정유형     1444 non-null   object 
 9   정액코드     1205 non-null   object 
 10  건유일      408 non-null    object 
 11  분만일      693 non-null    object 
 12  임신기간(일)  693 non-null    float64
 13  종부횟수     1444 non-null   int64  
 14  공태일수     400 non-null    float64
 15  분만간격(일)  400 non-null    float64
 16  초산일령     287 non-null    float64
dtypes: float64(5), int64(6), object(6)
memory usage: 203.1+ KB


In [131]:
len(breeding_data['개체번호'].unique())

294

### 환경 관리기

In [133]:
env_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경관리기정보.xlsx' , header=1)
env_data

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,No,지역,규모,농가명,장비종류,제조사아이디,장비번호,측정일시,측정값(℃ · % · ppm)
0,1,2.800000e+09,100두 미만,22902,습도,thekpmcorp,1,2022-12-31,67.59
1,2,2.800000e+09,100두 미만,22902,습도,thekpmcorp,1,2022-12-30,70.13
2,3,2.800000e+09,100두 미만,22902,습도,thekpmcorp,1,2022-12-29,58.50
3,4,2.800000e+09,100두 미만,22902,습도,thekpmcorp,1,2022-12-28,57.38
4,5,2.800000e+09,100두 미만,22902,습도,thekpmcorp,1,2022-12-27,70.79
...,...,...,...,...,...,...,...,...,...
16993,16994,NaN,500~1000,20268,온도,KILSOFT,1,2020-07-30,29.00
16994,16995,NaN,500~1000,20268,온도,KILSOFT,1,2020-07-29,28.00
16995,16996,NaN,500~1000,20268,온도,KILSOFT,1,2020-07-28,28.00
16996,16997,NaN,500~1000,20268,온도,KILSOFT,1,2020-07-27,30.00


In [140]:
# key로 사용 할 농가명이라는 column이 얼마나 겹치는지 확인
weight_farm_id = weight_data['농가명']
env_farm_id = env_data['농가명']
set(weight_farm_id) , set(weight_farm_id) & set(env_farm_id)

({20264, 20278, 20279, 21133}, {20264, 20278})

### 환경 제어기

겹치는 부분이 없어서 활용이 불가능함  

In [141]:
env_data = pd.read_excel('./raw_data/낙농_ICT데이터_환경제어기정보.xlsx' , header=1)
env_data

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,No,지역,규모,농가명,제조사아이디,장비번호,측정일시,측정값1(% · on/ff),측정값2,측정값3,측정값4,측정값5
0,1,4.100000e+09,100두 미만,21043,인프로,001,2022-12-31,0,0.0,NaN,NaN,NaN
1,2,4.100000e+09,100두 미만,21043,인프로,001,2022-12-29,5,0.0,NaN,NaN,NaN
2,3,4.100000e+09,100두 미만,21043,인프로,001,2022-12-25,0,0.0,NaN,NaN,NaN
3,4,4.100000e+09,100두 미만,21043,인프로,001,2022-12-24,0,0.0,NaN,NaN,NaN
4,5,4.100000e+09,100두 미만,21043,인프로,001,2022-12-19,0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9788,9789,NaN,300~400,21135,상욱축산,1,2021-12-09,19,111.0,NaN,NaN,NaN
9789,9790,NaN,300~400,21135,상욱축산,1,2021-12-08,19,112.0,NaN,NaN,NaN
9790,9791,NaN,300~400,21135,상욱축산,1,2021-12-07,19,112.0,NaN,NaN,NaN
9791,9792,NaN,300~400,21135,상욱축산,1,2021-12-06,19,111.0,NaN,NaN,NaN


In [142]:
# key로 사용 할 농가명이라는 column이 얼마나 겹치는지 확인
weight_farm_id = weight_data['농가명']
env_farm_id = env_data['농가명']
set(weight_farm_id) , set(weight_farm_id) & set(env_farm_id)

({20264, 20278, 20279, 21133}, set())

### TMR 

In [144]:
tmr_data = pd.read_excel('./raw_data/낙농_ICT데이터_TMR급이기정보.xlsx' , header=1)
tmr_data


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,No,지역,규모,농가명,제조사아이디,장비번호,측정일시,급이량(kg),사료코드,배합비율
0,1,4.100000e+09,100두 미만,20390,주식회사 실티,1,2021-05-08 22:00,1292.0,NaN,NaN
1,2,4.100000e+09,100두 미만,20390,주식회사 실티,1,2021-05-08 21:00,366.0,NaN,NaN
2,3,4.100000e+09,100두 미만,20390,주식회사 실티,1,2021-05-08 20:00,0.0,NaN,NaN
3,4,4.100000e+09,100두 미만,20390,주식회사 실티,1,2021-05-08 19:00,28.0,NaN,NaN
4,5,4.100000e+09,100두 미만,20390,주식회사 실티,1,2021-05-08 18:00,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
60395,60396,NaN,100~200,20250,주식회사 실티,1,2020-08-18 13:00,0.0,NaN,NaN
60396,60397,NaN,100~200,20250,주식회사 실티,1,2020-08-18 12:00,0.0,NaN,NaN
60397,60398,NaN,100~200,20250,주식회사 실티,1,2020-08-18 11:00,0.0,NaN,NaN
60398,60399,NaN,100~200,20250,주식회사 실티,1,2020-08-18 10:00,0.0,NaN,NaN


In [149]:
tmr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60400 entries, 0 to 60399
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       60400 non-null  int64  
 1   지역       53239 non-null  float64
 2   규모       60400 non-null  object 
 3   농가명      60400 non-null  int64  
 4   제조사아이디   60400 non-null  object 
 5   장비번호     60400 non-null  int64  
 6   측정일시     60400 non-null  object 
 7   급이량(kg)  60400 non-null  float64
 8   사료코드     0 non-null      float64
 9   배합비율     0 non-null      float64
dtypes: float64(4), int64(3), object(3)
memory usage: 4.6+ MB


In [145]:
# key로 사용 할 농가명이라는 column이 얼마나 겹치는지 확인
weight_farm_id = weight_data['농가명']
tmr_farm_id = tmr_data['농가명']
set(weight_farm_id) , set(weight_farm_id) & set(tmr_farm_id)

({20264, 20278, 20279, 21133}, {20264})

### 군사급이기 

In [147]:
# ICT 착유기
import os
import pandas as pd
from tqdm import tqdm_notebook

directory = './raw_data/군사급이기'
file_list = os.listdir(directory)
excel_files = [file for file in file_list if file.endswith('.xlsx')]

combined_data = pd.DataFrame()

for excel_file in tqdm_notebook(excel_files):
    file_path = os.path.join(directory, excel_file)
    data = pd.read_excel(file_path ,header=1)
    combined_data = combined_data.append(data, ignore_index=True)

combined_data


<ipython-input-147-8fe75600f1d2>:12: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/6 [00:00<?, ?it/s]

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-147-8fe75600f1d2>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-147-8fe75600f1d2>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

<ipython-input-147-8fe75600f1d2>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/npla

,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,섭취량(kg),설정량(kg),급이시작일자,급이종료일자
0,1,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-06 22:00,2.0,5.0,NaN,NaN
1,2,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-04 22:00,2.0,5.0,NaN,NaN
2,3,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-03 22:00,4.0,5.0,NaN,NaN
3,4,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-02 22:00,4.0,5.0,NaN,NaN
4,5,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-01 22:00,4.0,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
573924,69992,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-25 23:59,7.0,8.0,NaN,NaN
573925,69993,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-10 23:59,8.0,8.0,NaN,NaN
573926,69994,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-08 23:59,8.0,8.0,NaN,NaN
573927,69995,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-06 23:59,8.0,8.0,NaN,NaN


In [150]:
combined_data.to_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')

In [153]:
feed_data = pd.read_csv('./raw_data/낙농_ICT데이터_군사급이기정보.csv')
feed_data

<ipython-input-153-c4ca73013c4b>:1: DtypeWarning:

Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.



,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,섭취량(kg),설정량(kg),급이시작일자,급이종료일자
0,0,1,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-06 22:00,2.0,5.0,NaN,NaN
1,1,2,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-04 22:00,2.0,5.0,NaN,NaN
2,2,3,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-03 22:00,4.0,5.0,NaN,NaN
3,3,4,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-02 22:00,4.0,5.0,NaN,NaN
4,4,5,4.100000e+09,300~400,21134,20130105010033,드라발 (주),1,2023-02-01 22:00,4.0,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573924,573924,69992,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-25 23:59,7.0,8.0,NaN,NaN
573925,573925,69993,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-10 23:59,8.0,8.0,NaN,NaN
573926,573926,69994,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-08 23:59,8.0,8.0,NaN,NaN
573927,573927,69995,NaN,500~1000,20268,20170104010044,(주) 이지엠코리아,10,2022-08-06 23:59,8.0,8.0,NaN,NaN


In [154]:
feed_data = feed_data[feed_data['개체번호'].isin(weight_id)]
feed_data

,Unnamed: 0,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,섭취량(kg),설정량(kg),급이시작일자,급이종료일자
19517,19517,19518,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2023-05-31 22:00,8.0,9.0,0001-01-01 00:00,0001-01-01 00:00
19518,19518,19519,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2023-05-30 22:00,9.0,9.0,0001-01-01 00:00,0001-01-01 00:00
19519,19519,19520,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2023-05-29 22:00,8.0,9.0,0001-01-01 00:00,0001-01-01 00:00
19520,19520,19521,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2023-05-26 22:00,7.0,8.0,0001-01-01 00:00,0001-01-01 00:00
19521,19521,19522,4.400000e+09,300~400,21678,20160819010038,드라발 (주),1,2023-05-24 22:00,8.0,8.0,0001-01-01 00:00,0001-01-01 00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571996,571996,68064,NaN,300~400,20264,20180823020006,(주)애그리로보텍,1,2022-08-27 23:00,0.0,5.0,2022-08-27 00:00,2022-08-27 00:00
571997,571997,68065,NaN,300~400,20264,20180823020006,(주)애그리로보텍,1,2022-08-25 23:00,0.0,5.0,2022-08-25 00:00,2022-08-25 00:00
571998,571998,68066,NaN,300~400,20264,20180823020006,(주)애그리로보텍,1,2022-08-05 23:00,0.0,5.0,2022-08-05 00:00,2022-08-05 00:00
571999,571999,68067,NaN,300~400,20264,20180823020006,(주)애그리로보텍,1,2022-08-04 23:00,0.0,5.0,2022-08-04 00:00,2022-08-04 00:00


In [155]:
feed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56236 entries, 19517 to 572000
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  56236 non-null  int64  
 1   No          56236 non-null  int64  
 2   지역          1307 non-null   float64
 3   규모          56236 non-null  object 
 4   농가명         56236 non-null  int64  
 5   개체번호        56236 non-null  int64  
 6   제조사아이디      56236 non-null  object 
 7   장비번호        56236 non-null  int64  
 8   측정일시        56236 non-null  object 
 9   섭취량(kg)     56236 non-null  float64
 10  설정량(kg)     56236 non-null  float64
 11  급이시작일자      55965 non-null  object 
 12  급이종료일자      55965 non-null  object 
dtypes: float64(3), int64(5), object(5)
memory usage: 6.0+ MB


In [157]:
len(feed_data['개체번호'].unique())

201

In [159]:
feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_일반급이기정보.xlsx' , header=1)
feed_data


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,No,지역,규모,농가명,제조사아이디,장비번호,측정일시,급이량(kg)
0,1,4700000000,400~500,21143,DELAVAL,1,2021-10-20 22:00,7.3
1,2,4700000000,400~500,21143,DELAVAL,1,2021-10-19 22:00,7.3
2,3,4700000000,400~500,21143,DELAVAL,1,2021-10-18 22:00,7.2
3,4,4700000000,400~500,21143,DELAVAL,1,2021-10-17 22:00,7.2
4,5,4700000000,400~500,21143,DELAVAL,1,2021-10-16 22:00,6.8
5,6,4700000000,400~500,21143,DELAVAL,1,2021-10-15 22:00,7.3
6,7,4700000000,400~500,21143,DELAVAL,1,2021-10-14 22:00,7.0
7,8,4700000000,400~500,21143,DELAVAL,1,2021-10-13 22:00,7.3
8,9,4700000000,400~500,21143,DELAVAL,1,2021-10-12 22:00,7.2
9,10,4700000000,400~500,21143,DELAVAL,1,2021-10-04 22:00,7.2


In [160]:
feed_data = pd.read_excel('./raw_data/낙농_ICT데이터_착유군사급이기정보.xlsx' , header=1)
feed_data


/home/nplab/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,No,지역,규모,농가명,개체번호,제조사아이디,장비번호,측정일시,농후사료섭취량(kg),기타사료섭취량(kg)
0,1,4100000000,200~300,20264,20141204020102,(주)애그리로보텍,1,2022-12-31 00:00,7.0,0.0
1,2,4100000000,200~300,20264,20150322020008,(주)애그리로보텍,1,2022-12-31 00:00,6.0,0.0
2,3,4100000000,200~300,20264,20151207020121,(주)애그리로보텍,1,2022-12-31 00:00,5.0,0.0
3,4,4100000000,200~300,20264,20160615020009,(주)애그리로보텍,1,2022-12-31 00:00,2.0,0.0
4,5,4100000000,200~300,20264,20160620020037,(주)애그리로보텍,1,2022-12-31 00:00,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...
66,67,4100000000,200~300,20264,20201203010137,(주)애그리로보텍,1,2022-12-31 00:00,7.0,0.0
67,68,4100000000,200~300,20264,20201205010138,(주)애그리로보텍,1,2022-12-31 00:00,2.0,0.0
68,69,4100000000,200~300,20264,20201205010139,(주)애그리로보텍,1,2022-12-31 00:00,6.0,0.0
69,70,4100000000,200~300,20264,20210216010142,(주)애그리로보텍,1,2022-12-31 00:00,7.0,3.0
